In [1]:
pip install tenseal scikit-learn numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import tenseal as ts
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale data for better convergence
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict on raw (unencrypted) test data
y_pred = model.predict(X_test)
accuracy_plain = accuracy_score(y_test, y_pred)

print(f"Accuracy on raw (unencrypted) data: {accuracy_plain:.4f}")

Accuracy on raw (unencrypted) data: 0.9737


In [3]:
import tenseal as ts

# Create a TenSEAL BFV context with secret key enabled
bfv_context = ts.context(
    scheme=ts.SCHEME_TYPE.BFV,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60],
    plain_modulus=1032193  # Required for BFV
)

# Generate the required keys
bfv_context.generate_galois_keys()
bfv_context.generate_relin_keys()

# Keep the secret key so we can decrypt later
bfv_context.global_scale = 2**40  # Necessary for some operations
bfv_context.auto_rescale = True   # Helps maintain precision

In [4]:
# Encrypt the first test sample (scaled to integers)
bfv_test_sample = ts.bfv_vector(bfv_context, (X_test[0] * 1000).astype(int).tolist())

# Perform encrypted inference (dot product with model weights)
bfv_encrypted_prediction = model.coef_.dot(bfv_test_sample.decrypt(secret_key=bfv_context.secret_key())) + model.intercept_

# Convert prediction to probability
bfv_predicted_class = int((bfv_encrypted_prediction > 0).item())
print(f"BFV Encrypted Prediction: {bfv_predicted_class}, Actual Label: {y_test[0]}")


BFV Encrypted Prediction: 1, Actual Label: 1


In [5]:
def encrypted_accuracy(X_test, y_test, model, encryption_scheme):
    correct = 0
    for i in range(len(X_test)):
        if encryption_scheme == "BFV":
            enc_sample = ts.bfv_vector(bfv_context, (X_test[i] * 1000).astype(int).tolist())
            enc_prediction = model.coef_.dot(enc_sample.decrypt(secret_key=bfv_context.secret_key())) + model.intercept_

        predicted_class = int((enc_prediction > 0).item())  # Ensures conversion to scalar
        
        if predicted_class == y_test[i]:
            correct += 1
            
    return correct / len(y_test)

# Compute accuracy
bfv_accuracy = encrypted_accuracy(X_test, y_test, model, "BFV")

print(f"BFV Encrypted Accuracy: {bfv_accuracy:.4f}")


BFV Encrypted Accuracy: 0.9825
